In [9]:
# We import jsonfiy from Flask
import pandas as pd
from flask import Flask, url_for, request, jsonify, json
from joblib import dump, load
from werkzeug.exceptions import HTTPException
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import fr_core_news_sm
nlp = fr_core_news_sm.load()

main_path = '/Users/admin/Jedha/Fake_reviews_detection/1_Training_models/text_only_models'
#
lr = load(main_path + '/main_model.pkl')
#
preprocessor = load(main_path + '/preprocessor.pkl')
#
text_vectorizer = load(main_path + '/text_vectorizer.pkl')
#
topic_extractor = load(main_path + '/topic_extractor.pkl')


In [210]:
nlp = fr_core_news_sm.load()

In [261]:
review = review.replace('\n', ' ')
length = len(review)
exclam_count = len(''.join(ch for ch in review if ch =='!'))
uppercase_word_count = sum(map(str.isupper, review.split()))

meta = pd.DataFrame([[length, uppercase_word_count, exclam_count]], columns = ['len_review', 'exclam_count', 'upper_word_count'])

review_clean = re.sub(r"[^A-zÀ-ÿ0-9' ]+", " ", review).lower()
review_clean = re.sub(' +', ' ',review_clean).strip()
lemmatized = " ".join([token.lemma_ for token in nlp(review_clean) if token.lemma_ not in STOP_WORDS])

In [264]:
topic = pd.DataFrame(topic)

#####DOUBLE CHECK BINS

#Categorizing meta values, the bins are the quantiles used to cut the original dataset (see meta_data_analysis notebook)
#length
#meta['length'] = pd.cut(meta['len_review'], bins=[   5.,  368., 4998.], labels = ['low', 'high'], include_lowest=True)
#uppercase
meta['upper_word_count'] = pd.cut(meta['upper_word_count'].rank(method = 'first'), bins=[  0.,   3,   6, 147.], labels = ['low', 'mid', 'high'], include_lowest=True)
#exclam
meta['exclam_count'] = pd.cut(meta['exclam_count'].rank(method = 'first'), bins=[  0.,   0.01,   2., 133.],labels=['low', 'high', 'very_high'], include_lowest=True)

#We concatenate the topic and the metadata 
conc = pd.concat([topic, meta], axis = 1)

In [86]:
def get_prev(text):
    #Cleaning
    review = text.replace('\n', ' ')
    #Feature Engineering
    length = len(review)
    exclam_count = len(''.join(ch for ch in review if ch =='!'))
    uppercase_word_count = sum(map(str.isupper, review.split()))

    meta = pd.DataFrame([[length, exclam_count, uppercase_word_count]], columns = ['len_review', 'exclam_count', 'upper_word_count'])
    #Cleaning, Vectorization and Topic Extraction

    review_clean = re.sub(r"[^A-zÀ-ÿ0-9' ]+", " ", review).lower()
    review_clean = re.sub(' +', ' ',review_clean).strip()
    lemmatized = " ".join([token.lemma_ for token in nlp(review_clean) if token.lemma_ not in STOP_WORDS])

    vectrorized_item = text_vectorizer.transform([lemmatized])
    topic = topic_extractor.transform(vectrorized_item)

    topic = pd.DataFrame(topic)

    #####DOUBLE CHECK BINS

    #Categorizing meta values, the bins are the quantiles used to cut the original dataset (see meta_data_analysis notebook)
    #length
    meta['len_review'] = pd.cut(meta['len_review'], bins=[   5.,  368., 4998.], labels = ['low', 'high'], include_lowest=True)
    #uppercase
    meta['upper_word_count'] = pd.cut(meta['upper_word_count'], bins=[  0.,   1,   2, 147.], labels = ['low', 'mid', 'high'],           include_lowest=True)
    #exclam
    meta['exclam_count'] = pd.cut(meta['exclam_count'], bins=[  0.,   1,   3., 133.],labels=['low', 'high', 'very_high'], include_lowest=True)

    #We concatenate the topic and the metadata 
    conc = pd.concat([topic, meta], axis = 1)

    #preprocessing 
    data = preprocessor.transform(conc)
    prev = lr.predict(data)
    proba = lr.predict_proba(data)

    return prev[0], proba[0][0], proba[0][1], conc.iloc[-3:]


In [89]:
conc

NameError: name 'conc' is not defined

In [2]:
ls

Model_SVD_Text_and_Text_metadata.ipynb
Model_SVD_Text_and_metadata.ipynb
Model_Text_and_text_metadata_tensorflow.ipynb
__pycache__/
app.py
getprev.py
model_files/
templates/


In [3]:
from getprev import get_prev as pp

In [88]:
get_prev('ALORS LA CEST TROP !!!!')

(1,
 0.10376780326019652,
 0.8962321967398035,
      0    1    2    3    4    5    6    7    8    9  ...  493  494  495  496  \
 0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
 
    497  498  499  len_review  exclam_count  upper_word_count  
 0  0.0  0.0  0.0         low     very_high              high  
 
 [1 rows x 503 columns])

In [85]:
french_reviews.iloc[-3:]

,text_review,is_fake_review,text_review_clean
90594,Très déçu!!!\nCe soir j'ai eu envie de manger ...,1,tre decu soir envie manger chicken tika grand ...
90595,J'y vais depuis le début mais j'avoue qu'avec ...,1,y aller debut avoue desormai bon viande faite ...
90596,Meilleur grec dans les environs + personnel au...,1,meilleur grec personnel top superrbe hygiene a...
